In [1]:
import os
import time
import pandas as pd
from datetime import datetime
import gender_guesser.detector as gg


def cleanData(data, scrapeDate):
    # Remove ongoing projects (at the time of scraping)
    scrapeDateUNIX = time.mktime(datetime(scrapeDate[0], scrapeDate[1], scrapeDate[2], 12, 0, 0).timetuple())  # conversion to UNIX timestamp
    data = data[(data['deadline'] < scrapeDateUNIX)]

    # Modifying columns
    data['goal'] *= data['fx_rate']
    data['raised'] = data['pledged'] * data['fx_rate']
    data['country'] = data['country'].apply(lambda entry: getCountry(entry))

    # Creating new columns
    data['projectID'] = data['profile'].apply(lambda entry: getProjectID(entry))
    data['successful'] = (data['raised'] >= data['goal'])
    data['disaster'] = (data['raised'] == 0)
    data['duration'] = ((data['deadline'] - data['launched_at']) / 3600 / 24)
    data['creator_gender'] = data[['country', 'creator']].apply(lambda entry: getGender(entry), axis=1)
    data['U.S.'] = (data['country'] == "usa")
    data['url'] = data['urls'].apply(lambda entry: getUrl(entry))

    # Remove entries with an 'unknown' or 'andy' gender
    data = data[(data['creator_gender'].isin(['male', 'female', 'mostly_male', 'mostly_female']))]

    # Modifying columns, again
    data = data.set_index('projectID')
    data['successful'] = data['successful'].replace({True: 1, False: 0})
    data['disaster'] = data['disaster'].replace({True: 1, False: 0})
    data['creator_gender'] = data['creator_gender'].replace({'male': 1, 'female': 0, 'mostly_male': 1, 'mostly_female': 0})
    data['U.S.'] = data['U.S.'].replace({True: 1, False: 0})

    # Misc
    data = data.rename(columns={"backers_count": "no_backers"})
    data = data[['successful', 'disaster', 'goal', 'raised', 'no_backers', 'duration', 'creator_gender', 'U.S.', 'url']]
    return data

def getCountry(countryCode):
    if countryCode in countries.keys():
        return countries.get(countryCode)
    else:
        return "other_countries"

def getProjectID(entry):
    projectID = entry.split(",")[1]
    projectID = int(projectID.split(":")[1])
    return projectID

def getGender(entry):
    fullName = entry['creator'].split(",")[1]
    fullName = fullName.split(":")[1]
    fullName = fullName.replace("\"", "")
    firstName = fullName.split(" ")[0]
    gender = genderDetector.get_gender(firstName, entry['country'])
    return gender

def getUrl(entry):
    url = entry.split(",")[0]
    url = url.split("\"")[5]
    url = url.split("?")[0]
    return url

def getDateFromFileName(fileName):
    split = fileName.split('-')
    del split[2]
    return '-'.join(split)

scrapeDates = [
    [2019, 6, 13], [2019, 7, 18], [2019, 8, 15], [2019, 9, 12], [2019, 10, 17], [2019, 11, 14], [2019, 12, 12],
    [2020, 1, 16], [2020, 2, 13], [2020, 3, 12], [2020, 4, 16], [2020, 5, 14], [2020, 6, 18], [2020, 7, 16], [2020, 8, 13], [2020, 9, 17], [2020, 10, 15], [2020, 11, 12], [2020, 12, 17],
    [2021, 1, 14], [2021, 2, 11], [2021, 3, 18], [2021, 4, 15], [2021, 5, 17], [2021, 6, 17], [2021, 7, 15], [2021, 8, 12], [2021, 9, 16], [2021, 10, 15], [2021, 11, 19], [2021, 12, 14],
    [2022, 1, 20], [2022, 2, 10], [2022, 3, 24], [2022, 4, 21], [2022, 5, 19], [2022, 6, 9], [2022, 7, 14], [2022, 8, 11]
]
countries = {
    "GB": "great_britain", "IE": "ireland", "US": "usa", "IT": "italy", "MT": "malta", "PT": "portugal", "ES": "spain", "FR": "france",
    "BE": "belgium", "LU": "luxembourg", "NL": "the_netherlands", "DE": "germany", "AT": "austria", "CH": "swiss", "IS": "iceland",
    "DK": "denmark", "NO": "norway", "SE": "sweden", "FI": "finland", "EE": "estonia", "LV": "latvia", "LT": "lithuania", "PL": "poland",
    "CZ": "czech_republic", "SK": "slovakia", "HU": "hungary", "RO": "romania", "BG": "bulgaria", "BA": "bosniaand", "HR": "croatia",
    "XK": "kosovo", "MK": "macedonia", "ME": "montenegro", "RS": "serbia", "SI": "slovenia", "AL": "albania", "GR": "greece", "RU": "russia",
    "BY": "belarus", "MD": "moldova", "UA": "ukraine", "AM": "armenia", "AZ": "azerbaijan", "GE": "georgia", "KZ": "the_stans", "KG": "the_stans",
    "TJ": "the_stans", "TM": "the_stans", "UZ": "the_stans", "TR": "turkey", "SA": "arabia", "IL": "israel", "CN": "china", "IN": "india",
    "JP": "japan", "KR": "korea", "VN": "vietnam"
}
files = os.listdir("Kickstarter Data")
genderDetector = gg.Detector()

allData = cleanData(pd.read_csv("Kickstarter Data\\" + files[0]), scrapeDates[0])

previousDate = getDateFromFileName(files.pop(0))
scrapeDate = scrapeDates.pop(0)
for file in files:
    date = getDateFromFileName(file)
    if date != previousDate:
        previousDate = date
        scrapeDate = scrapeDates.pop(0)

    allData = pd.concat([allData, cleanData(pd.read_csv("Kickstarter Data\\" + file), scrapeDate)])

allData

C:\Users\Perso\AppData\Local\Temp/ipykernel_16252/2388108584.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['goal'] *= data['fx_rate']
C:\Users\Perso\AppData\Local\Temp/ipykernel_16252/2388108584.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['raised'] = data['pledged'] * data['fx_rate']
C:\Users\Perso\AppData\Local\Temp/ipykernel_16252/2388108584.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] =

,successful,disaster,goal,raised,no_backers,duration,creator_gender,U.S.,url
projectID,,,,,,,,,
2246288,1,0,3400.0,18559.00,740,31.003067,1,1,https://www.kickstarter.com/projects/skullgard...
2944476,0,0,4000.0,80.00,3,58.959757,1,1,https://www.kickstarter.com/projects/203179337...
1985482,1,0,30000.0,135589.94,1752,29.958333,1,1,https://www.kickstarter.com/projects/madoverlo...
2309240,1,0,4000.0,4652.00,185,30.000000,1,1,https://www.kickstarter.com/projects/672576444...
3398789,1,0,4900.0,5630.00,111,30.003738,0,1,https://www.kickstarter.com/projects/pabkins/g...
...,...,...,...,...,...,...,...,...,...
912317,0,1,10000.0,0.00,0,30.000000,1,1,https://www.kickstarter.com/projects/207721367...
1229980,0,1,5000.0,0.00,0,29.061620,1,1,https://www.kickstarter.com/projects/348446415...
1136803,0,1,3000.0,0.00,0,29.000000,0,1,https://www.kickstarter.com/projects/179067868...
